In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers
from wimbledon.github import preferences_availability as pref

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()  # use get token here

GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.5134966019541025 
Enpoint: people/475627 | Time: 0.46882185200229287 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.4810615499736741 
PROJECTS
Enpoint: projects | Time: 0.8904391510295682 
ROLES
Enpoint: roles | Time: 0.6239916689810343 
PEOPLE
Enpoint: people | Time: 0.6070703989826143 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.48605926701566204 
MILESTONES
Enpoint: milestones | Time: 0.44577634998131543 
ASSIGNMENTS
Enpoint: assignments | Time: 0.7773829990183003 
DONE! (5.4s)


REG team
=====

We need to be able to get a team members availability for specific date ranges:

In [5]:
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
print(pref.get_person_availability(fc, "Ed Chalstrey", start_date, end_date))
print(pref.get_person_availability(fc, 458589, start_date, end_date))

0.46
1.0


Projects should have a consistent resource required number over the months they aren't 0, but may be useful to look at the resource requirement over a specific date range:

In [6]:
print(pref.get_project_requirement(fc, "CHANCE", start_date, end_date))
print(pref.get_project_requirement(fc, 2061048, start_date, end_date))

0
0.23


Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [7]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [8]:
preference_data_df = pref.get_preference_data(fc, token, emoji_mapping=emoji_mapping)
preference_data_df

,Sargasso (Intel Sparse),DetectorChecker,HSBC EDS,AI for City Planning,Reproducible Research,PDQ,Crypto Federated Learning (SHEEP),The Turing Way,Fitbit classification,Parallel Monte Carlo,...,Census 2021 backbrief,Solar Panel Detection,NoCell - Phase 2,Hate Speech,SAIL multimorbidity,QUIPP - Synthetic data,WAYS WP1 - Extending BackFillz,WAYS WP2 - Auto-generating precursor visualisations,Survival outcome - Roche,British Airways Ancillaries management
Person,,,,,,,,,,,,,,,,,,,,,
Oliver Strickson,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,👍,👍,✅,❌,👍,👍,👍,👍,❓,👍
Evelina Gabasova,❓,❓,❓,❓,❓,❓,❓,✅,👍,👍,...,❓,👍,👍,👍,👍,👍,👍,👍,❓,❌
Giovanni Colavizza,❓,❓,❓,❓,❓,❓,❓,👍,❌,❌,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
James Geddes,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,👍,👍,✅,❓,❓,❓,❓,❓,❓,👍
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Louise Bowler,❓,❓,❓,❓,❓,❓,❓,✅,❌,❌,...,👍,👍,❌,👍,👍,👍,👍,👍,❓,✅
May Yong,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❌,❓,❌,👍,❓,❌,👍,👍,❓,❓
Nick Barlow,❓,❓,❓,❓,❓,❓,❓,❌,👍,❌,...,❌,👍,👍,👍,👍,👍,👍,👍,❓,❌
Radka Jersakova,❓,❓,❓,❓,❓,❓,❓,👍,❌,❌,...,👍,❓,❌,👍,👍,👍,❌,❌,❓,❓


In [9]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [12]:
preferences = pref.get_preferences(fc, preference_data_df, emojis_only=True)

In [14]:
preferences["Census 2021 backbrief"]["Ed Chalstrey"]

'👍'

In [15]:
# preferences["CHANCE"]["Ed Chalstrey"]

In [16]:
css = """<style>
.tableFixHead {
          overflow: scroll;
          max-height: 100%;
          max-width: 100%;
        }
thead th {
          position: sticky;
          top: 0;
        }
tbody th {
          position: sticky;
          left: 0;
        }
thead th:first-child {
          left: 0;
          z-index: 1;
}
</style>"""
emoji_table = preferences.to_html()
html_table = css + """<div class="tableFixHead">""" + emoji_table + """</div>"""
# print(html_table)
with open("preferences.html", "w") as preferences_file:
    preferences_file.write(html_table)
display(HTML(html_table))

,All in One Cancer imaging optimisation,Auto DL,Census 2021 backbrief,DFTG Commons WP 3.4/3.5,Interpretable AI for health,Learning Machines,Living With Machines,Logics for DS,Machine learning for cancer,Micro-location analytics,NoCell - Phase 2,Probabilistic FEM,SAIL multimorbidity,Safe Haven,Scaleable Monte Carlo,Solar Panel Detection,UQM^3,Uncertainty in Government Modelling,Urban Agriculture,Urban systems resilience,Visualisation: Roy Ruddle TF project,Wrattler 2019-2020
Person,,,,,,,,,,,,,,,,,,,,,,
Ashwini Venkatasubramaniam,👍,❌,❌,❌,✅,❓,❓,❌,❌,👍,❌,❓,👍,❓,❌,❌,❓,❓,❌,❓,❌,❓
Camila Rangel Smith,👍,✅,👍,❌,👍,❓,👍,👍,✅,✅,❌,❌,👍,❓,❌,👍,👍,👍,❌,👍,❓,❓
Ed Chalstrey,✅,❌,👍,❓,👍,❓,❓,❌,✅,✅,❌,❓,👍,❓,👍,✅,❓,❓,👍,❌,❌,❓
Eric Daub,❌,❌,❓,👍,❌,❓,❓,❌,❌,❌,❌,❓,❌,❓,👍,👍,❓,❓,👍,👍,❌,❓
Evelina Gabasova,👍,👍,❓,❌,👍,❓,👍,👍,👍,👍,👍,❌,👍,❓,❌,👍,❌,👍,❌,👍,❌,❓
Giovanni Colavizza,❓,👍,❓,❓,❌,❓,✅,👍,❌,❌,❓,❓,❓,❓,❓,❓,❌,❌,❓,❌,❌,❓
Jack Roberts,👍,👍,👍,❌,👍,❓,✅,❌,👍,👍,❌,👍,👍,❓,❓,✅,❌,❓,✅,👍,❌,❓
James Geddes,❓,❓,👍,❓,❓,❓,❓,❓,❓,❓,✅,❓,❓,❓,👍,👍,❓,❓,👍,❓,❓,❓
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,✅,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓


In [17]:
preferences_with_availability = pref.get_preferences(fc, preference_data_df)

In [18]:
emoji_table2 = preferences_with_availability.to_html()
display(HTML(emoji_table2))

,All in One Cancer imaging optimisation,Auto DL,Census 2021 backbrief,DFTG Commons WP 3.4/3.5,Interpretable AI for health,Learning Machines,Living With Machines,Logics for DS,Machine learning for cancer,Micro-location analytics,NoCell - Phase 2,Probabilistic FEM,SAIL multimorbidity,Safe Haven,Scaleable Monte Carlo,Solar Panel Detection,UQM^3,Uncertainty in Government Modelling,Urban Agriculture,Urban systems resilience,Visualisation: Roy Ruddle TF project,Wrattler 2019-2020
Person,,,,,,,,,,,,,,,,,,,,,,
Ashwini Venkatasubramaniam,👍 23.0% (0.23 / 1.0),❌ 34.0% (0.17 / 0.5),❌ 0.0% (0.0 / 0.28),❌ 8.25% (0.33 / 4.0),✅ 50.0% (0.5 / 1.0),❓ 35.97% (0.5 / 1.39),❓ 27.78% (0.2 / 0.72),❌ 136.0% (0.34 / 0.25),❌ 43.0% (0.43 / 1.0),👍 12.0% (0.12 / 1.0),❌ 100.0% (0.5 / 0.5),❓ 80.0% (0.4 / 0.5),👍 0.0% (0.0 / 0.5),❓ 12.12% (0.16 / 1.32),❌ 8.0% (0.08 / 1.0),❌ 84.0% (0.42 / 0.5),❓ 100.0% (0.5 / 0.5),❓ 88.0% (0.44 / 0.5),❌ 66.0% (0.33 / 0.5),❓ 33.0% (0.33 / 1.0),❌ 36.0% (0.36 / 1.0),❓ 50.0% (0.5 / 1.0)
Camila Rangel Smith,👍 100.0% (1.0 / 1.0),✅ 16.0% (0.08 / 0.5),👍 0.0% (0.0 / 0.28),❌ 25.0% (1.0 / 4.0),👍 100.0% (1.0 / 1.0),❓ 58.27% (0.81 / 1.39),👍 138.89% (1.0 / 0.72),👍 264.0% (0.66 / 0.25),✅ 64.0% (0.64 / 1.0),✅ 100.0% (1.0 / 1.0),❌ 200.0% (1.0 / 0.5),❌ 100.0% (0.5 / 0.5),👍 200.0% (1.0 / 0.5),❓ 18.94% (0.25 / 1.32),❌ 100.0% (1.0 / 1.0),👍 116.0% (0.58 / 0.5),👍 200.0% (1.0 / 0.5),👍 138.0% (0.69 / 0.5),❌ 84.0% (0.42 / 0.5),👍 17.0% (0.17 / 1.0),❓ 50.0% (0.5 / 1.0),❓ 100.0% (1.0 / 1.0)
Ed Chalstrey,✅ 25.0% (0.25 / 1.0),❌ 0.0% (0.0 / 0.5),👍 0.0% (0.0 / 0.28),❓ 8.25% (0.33 / 4.0),👍 100.0% (1.0 / 1.0),❓ 33.09% (0.46 / 1.39),❓ 30.56% (0.22 / 0.72),❌ 128.0% (0.32 / 0.25),✅ 43.0% (0.43 / 1.0),✅ 0.0% (0.0 / 1.0),❌ 134.0% (0.67 / 0.5),❓ 40.0% (0.2 / 0.5),👍 0.0% (0.0 / 0.5),❓ 10.61% (0.14 / 1.32),👍 0.0% (0.0 / 1.0),✅ 66.0% (0.33 / 0.5),❓ 158.0% (0.79 / 0.5),❓ 100.0% (0.5 / 0.5),👍 34.0% (0.17 / 0.5),❌ 0.0% (0.0 / 1.0),❌ 29.0% (0.29 / 1.0),❓ 67.0% (0.67 / 1.0)
Eric Daub,❌ 83.0% (0.83 / 1.0),❌ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.28),👍 19.75% (0.79 / 4.0),❌ 50.0% (0.5 / 1.0),❓ 33.09% (0.46 / 1.39),❓ 118.06% (0.85 / 0.72),❌ 168.0% (0.42 / 0.25),❌ 29.0% (0.29 / 1.0),❌ 100.0% (1.0 / 1.0),❌ 116.0% (0.58 / 0.5),❓ 40.0% (0.2 / 0.5),❌ 200.0% (1.0 / 0.5),❓ 8.33% (0.11 / 1.32),👍 100.0% (1.0 / 1.0),👍 50.0% (0.25 / 0.5),❓ 100.0% (0.5 / 0.5),❓ 62.0% (0.31 / 0.5),👍 34.0% (0.17 / 0.5),👍 0.0% (0.0 / 1.0),❌ 21.0% (0.21 / 1.0),❓ 50.0% (0.5 / 1.0)
Evelina Gabasova,👍 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.28),❌ 12.5% (0.5 / 4.0),👍 50.0% (0.5 / 1.0),❓ 27.34% (0.38 / 1.39),👍 69.44% (0.5 / 0.72),👍 128.0% (0.32 / 0.25),👍 29.0% (0.29 / 1.0),👍 50.0% (0.5 / 1.0),👍 100.0% (0.5 / 0.5),❌ 40.0% (0.2 / 0.5),👍 100.0% (0.5 / 0.5),❓ 8.33% (0.11 / 1.32),❌ 50.0% (0.5 / 1.0),👍 50.0% (0.25 / 0.5),❌ 100.0% (0.5 / 0.5),👍 62.0% (0.31 / 0.5),❌ 34.0% (0.17 / 0.5),👍 0.0% (0.0 / 1.0),❌ 21.0% (0.21 / 1.0),❓ 50.0% (0.5 / 1.0)
Giovanni Colavizza,❓ 0.0% (0.0 / 1.0),👍 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.28),❓ 0.0% (0.0 / 4.0),❌ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 1.39),✅ 0.0% (0.0 / 0.72),👍 0.0% (0.0 / 0.25),❌ 0.0% (0.0 / 1.0),❌ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 1.32),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 0.5),❌ 0.0% (0.0 / 0.5),❌ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.5),❌ 0.0% (0.0 / 1.0),❌ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 1.0)
Jack Roberts,👍 67.0% (0.67 / 1.0),👍 66.0% (0.33 / 0.5),👍 178.57% (0.5 / 0.28),❌ 12.5% (0.5 / 4.0),👍 50.0% (0.5 / 1.0),❓ 35.97% (0.5 / 1.39),✅ 97.22% (0.7 / 0.72),❌ 168.0% (0.42 / 0.25),👍 50.0% (0.5 / 1.0),👍 67.0% (0.67 / 1.0),❌ 100.0% (0.5 / 0.5),👍 100.0% (0.5 / 0.5),👍 134.0% (0.67 / 0.5),❓ 18.94% (0.25 / 1.32),❓ 71.0% (0.71 / 1.0),✅ 100.0% (0.5 / 0.5),❌ 100.0% (0.5 / 0.5),❓ 100.0% (0.5 / 0.5),✅ 100.0% (0.5 / 0.5),👍 50.0% (0.5 / 1.0),❌ 50.0% (0.5 / 1.0),❓ 50.0% (0.5 / 1.0)
James Geddes,❓ 47.0% (0.47 / 1.0),❓ 4.0% (0.02 / 0.5),👍 0.0% (0.0 / 0.28),❓ 8.75% (0.35 / 4.0),❓ 10.0% (0.1 / 1.0),❓ 17.27% (0.24 / 1.39),❓ 68.06% (0.49 / 0.72),❓ 92.0% (0.23 

In [19]:
pref.get_preferences(fc, preference_data_df, person="Ed Chalstrey", positive_only=True)

,All in One Cancer imaging optimisation,Census 2021 backbrief,Interpretable AI for health,Machine learning for cancer,Micro-location analytics,SAIL multimorbidity,Scaleable Monte Carlo,Solar Panel Detection,Urban Agriculture
Person,,,,,,,,,
Ed Chalstrey,✅ 25.0% (0.25 / 1.0),👍 0.0% (0.0 / 0.28),👍 100.0% (1.0 / 1.0),✅ 43.0% (0.43 / 1.0),✅ 0.0% (0.0 / 1.0),👍 0.0% (0.0 / 0.5),👍 0.0% (0.0 / 1.0),✅ 66.0% (0.33 / 0.5),👍 34.0% (0.17 / 0.5)


In [20]:
pref.get_preferences(fc, preference_data_df, project="All in One Cancer imaging optimisation")

,All in One Cancer imaging optimisation
Person,
Ashwini Venkatasubramaniam,👍 23.0% (0.23 / 1.0)
Camila Rangel Smith,👍 100.0% (1.0 / 1.0)
Ed Chalstrey,✅ 25.0% (0.25 / 1.0)
Eric Daub,❌ 83.0% (0.83 / 1.0)
Evelina Gabasova,👍 50.0% (0.5 / 1.0)
Giovanni Colavizza,❓ 0.0% (0.0 / 1.0)
Jack Roberts,👍 67.0% (0.67 / 1.0)
James Geddes,❓ 47.0% (0.47 / 1.0)
James Hetherington,❓ 9.0% (0.09 / 1.0)


In [21]:
pref.get_preferences(fc, preference_data_df, project="All in One Cancer imaging optimisation", positive_only=True)

ValueError: arrays must all be same length